In [1]:
### NN Predictor with prior distribution (or recency) sampling ###
##################################################################


In [2]:
#load data, condensed as we did this previously
def load_clinical_eeg_data(datapath, sub):
    import pandas as pd
    alldata = pd.read_csv(os.path.join(datapath, sub + '.csv')) #removed 'train' bc of how I saved
    alldata.rename(columns={'Unnamed: 0': 'Index'})
    eegevents = alldata[['labels', 'chunks']]
    alldata.drop(['Unnamed: 0', 'labels', 'chunks'], axis=1, inplace=True)
    names = alldata.keys()
    return alldata.iloc[:].as_matrix(), eegevents, names
import os
import numpy as np
os.chdir("C:\\Users\\adam1brownell\\Desktop\Winter2017\\188B Files\Project")
path = os.getcwd()

#Get subject names from appropriate dir, -4 for .csv suffux
subjects = [f[:-4] for f in os.listdir(path)]

data, label_chunk, nodes = load_clinical_eeg_data(path,subjects[3])

In [3]:
# Relabel the data so that we can train our model properly
import numpy as np
label = np.array(label_chunk)[:,0]
#For now...
pLabel = label

In [4]:
#Split data into train/test
# data = data, pLabel = labels
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, pLabel, test_size=.2)


#Specify Hidden Layers
#There are 23 nodes, so input could be 23 nodes (or smaller) and output is 3 nodes
#Hidden layers are 10 and 5, like keras intro stated
layer_sizes = [10, 5]

In [5]:
#Build NN
from keras.models import Sequential
NNmodel = Sequential()

#Add layers
from keras.layers.core import Dense, Activation

#Input Layer -> 1
NNmodel.add(Dense(input_dim=X_train.shape[1], output_dim=layer_sizes[0]))
NNmodel.add(Activation('sigmoid'))
#Hidden Layer 1 -> 2
NNmodel.add(Dense(input_dim=layer_sizes[0], output_dim=layer_sizes[1]))
NNmodel.add(Activation('sigmoid'))
#Hidden Layer 2 -> output
NNmodel.add(Dense(output_dim=np.unique(y_train).shape[0], init='uniform'))
NNmodel.add(Activation('softmax'))

# Next we let the network know how to learn
#(From Keras doc)
from keras.optimizers import SGD
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
NNmodel.compile(loss='categorical_crossentropy', optimizer=sgd)

# Before we can fit the network, we have to one-hot vectorize our response.
# Fortunately, there is a keras method for that.
from keras.utils.np_utils import to_categorical
y_train_vectorized = to_categorical(y_train)


Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [6]:
#Fit Model
NNmodel.fit(X_train, y_train_vectorized, nb_epoch=3, batch_size=50)
#NNmodel.fit(X_train, y_train_vectorized, nb_epoch=1000, batch_size=50)

Epoch 1/3
258969/258969 [==============================] - 118s - loss: 0.3016   
Epoch 2/3
258969/258969 [==============================] - 116s - loss: 0.2957   
Epoch 3/3
258969/258969 [==============================] - 117s - loss: 0.2957   


In [7]:
#probability of 1 sample
NNmodel.predict_proba(np.reshape(X_test[1,:], (1,23)))

1/1 [==============================] - 0s


array([[ 0.91337603,  0.08662394]], dtype=float32)

In [8]:
#score model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, NNmodel.predict_classes(X_test))

64192/64743 [============================>.] - ETA: 0s

0.91268554129403956

In [9]:
#I want to take the prob of it being before a seizure and add it to a distribution to find the true probability
def ten_sec(model,datapoint,prior,i):
    #This is the NN predict without considering time context:
    prop = model.predict_proba(datapoint)
    #Augment prediction based on Context
    
    prior[i] = prop
    #skepticism:
    skept_prop = np.mean(prior)
    return skept_prior
    #If previous mean AND current prediction are above threshold, then higher probability

In [10]:
#Get predict_proba for all datapoints
#For every data point
#If the previous prediction agrees with current point:
    #Augment probability by "congruency value"
#Else
    #Augement probability by "discongruency vale"

In [11]:
#This part shows that despite getting 92% accuracy, we are still only at chance guessing
#This is because our NN learns to always guess "no eizure" since seizures are so rare

proba = NNmodel.predict_proba(X_test, batch_size=32)
classes = np.argmax(proba, axis=1)
import sklearn.metrics as met
res = NNmodel.predict(X_test)
met.roc_auc_score(y_test, classes)

64096/64743 [============================>.] - ETA: 0s

0.5

In [12]:
#Retesting the NN with the max freq per second
def take_max_every_64_points(x):
    i=0; 
    mean_array=[]
    while(i<(x.shape[0])):
        array=x[i:i+65]
        mean_array.append(np.max(array))
        i+=64
    mean_array=np.array(mean_array)
    return mean_array
dataset20 = np.apply_along_axis(take_max_every_64_points,0,data)
y20=take_max_every_64_points(np.array(label_chunk)[:,0])

X_train, X_test, y_train, y_test = train_test_split(dataset20, y20, test_size=.2)
y_train_vectorized = to_categorical(y_train)

NNmodel.fit(X_train, y_train_vectorized, nb_epoch=3, batch_size=50)
accuracy_score(y_test, NNmodel.predict_classes(X_test))

Epoch 1/3
4046/4046 [==============================] - 1s - loss: 0.2999     
Epoch 2/3
4046/4046 [==============================] - 1s - loss: 0.2999     
Epoch 3/3
 480/1012 [=============>................] - ETA: 0s

0.91600790513833996

In [13]:
#Still at chance...
proba = NNmodel.predict_proba(X_test, batch_size=32)
classes = np.argmax(proba, axis=1)
import sklearn.metrics as met
res = NNmodel.predict(X_test)
met.roc_auc_score(y_test, classes)

 576/1012 [================>.............] - ETA: 0s

0.5

In [14]:
# replacing chunk numbers with their indexes
chunks = np.array(label_chunk)[:,1]
labels = np.array(label_chunk)[:,0]


chunks_numbers = np.unique(chunks, return_index=True)

i = 1
new_chunks = []
total_chunks = chunks_numbers[0].shape[0]

while i < total_chunks:
    for j in range(chunks_numbers[1][i-1], chunks_numbers[1][i]):
        new_chunks.append(i-1)
    i += 1
for j in range(chunks_numbers[1][total_chunks-1], chunks.shape[0]):
    new_chunks.append(i-1)
new_chunks = np.array(new_chunks)
# print new_chunks.shape
# chunks_numbers = np.unique(new_chunks, return_index=True)
# print chunks_numbers
chunks = new_chunks


# creating new labels for seizure, pre- and post- seizure
# in each chunk - 10 minutes before/after sezure = 600 samples X 599
# -1 = pre, 1 = seizure, 0 = post

#print labels.shape
chunks_numbers = np.unique(chunks, return_index=True)
#(array([0, 1, 2, 3, 4, 5]), array([   0, 1203, 2430, 3670, 4921, 6211]))

i = 1
while i < total_chunks:
    start = chunks_numbers[1][i-1]
    end = chunks_numbers[1][i]-1
    for j in range(start, end):
        labels[j] = -1
        if labels[j+1] == 1:
            break
    i += 1

start = end + 1
end = labels.shape[0]
for j in range(start, end):
    labels[j] = -1
    if labels[j+1] == 1:
        break

In [15]:
# chunks_numbers[1] = chunk cutoffs
# total_chunks = number of chunks
chunks_numbers[1]

array([     0,  72448, 156288, 239232], dtype=int64)

In [16]:
def re_label(chunks,labels):
    pLabel = [0 for i in range(len(labels))]
    
    #For every chunk
    for i in range(total_chunks):
        
        #If on the last chunk set limit to end of all data, else set to beginning of next chunk
        if i == 3:
            limit = len(labels)
        else:
            limit = chunks_numbers[1][i+1]

        #flag for seizure occuring now, set to false
        seiz_rn = False


        #Looking at all values within a chunk
        for datapoint in range(i,limit):

            #If no seizure rn and none before, add 0
            if labels[datapoint] == 0 and seiz_rn == False:
                pLabel[datapoint] = 0 #I don't need this since it already is set to 0 but kept for readibility

            #If seizure rn and yes before, add 1
            elif labels[datapoint] == 1 and seiz_rn == True:
                pLabel[datapoint] = 1

            #If seizure rn and none before, add 1 AND change the last 640 (10sec) of pLabel to 2
            elif labels[datapoint] == 1 and seiz_rn == False:
                for j in range(datapoint-640,datapoint+1):
                    pLabel[j] = 2
                seiz_rn = True

            #If label is 0 and seiz is true, seiz just ended
            else:
                pLabel[datapoint] = 0 #Don't need, just for reading
                seiz = False
                
    return pLabel

def re_label2(chunks,labels):
    pLabel = [0 for i in range(len(labels))]
    seiz_rn = False
    for datapoint in range(len(labels)):
        #If no seizure rn and none before, add 0
        if labels[datapoint] == 0 and seiz_rn == False:
            pLabel[datapoint] = 0 #I don't need this since it already is set to 0 but kept for readibility

        #If seizure rn and yes before, add 1
        elif labels[datapoint] == 1 and seiz_rn == True:
                pLabel[datapoint] = 1

        #If seizure rn and none before, add 1 AND change the last 640 (10sec) of pLabel to 2
        elif labels[datapoint] == 1 and seiz_rn == False:
            for j in range(datapoint-640,datapoint+1):
                pLabel[j] = 2
            seiz_rn = True

        #If label is 0 and seiz is true, seiz just ended
        else:
            pLabel[datapoint] = 0 #Don't need, just for reading
            seiz = False
    
    return pLabel        
    

In [20]:
pLabel = re_label2(chunks,labels)